In [1]:
import requests
from bs4 import BeautifulSoup
import datetime
import pandas as pd
from tqdm import tqdm
import regex as re
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from matplotlib.lines import Line2D

import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output

In [2]:
with open('/Users/euan_brown/Documents/GitHub/NCAA-fh-project/results.pkl', 'rb') as f:
    df = pickle.load(f)

In [3]:
# get home team and away team names

df['home']= df.df_sum.apply(lambda x: x[0][1][:x[0][1].find('(') - 1])
df['away']= df.df_sum.apply(lambda x: x[0][2][:x[0][2].find('(') - 1])

def clean_team_name(team_name):
    cleaned_name = re.sub(r'^#\d+\s*', '', team_name)
    return cleaned_name.strip()

df['home'] = df['home'].apply(clean_team_name)
df['away'] = df['away'].apply(clean_team_name)

In [4]:
# add column names to box scores

# list names
box_cols = ['name','pos','goals','ast','shatt',
            'sog','fouls','rc','yc','gc',
            'ggp','ggs','min','ga','sv',
            'sho','g_wins','g_loss','dsv',
            'corners','ps','psa'
            ]

# construct dict
cols = dict(zip(range(0,20),box_cols))

# lambda apply
df.df_home = df.df_home.apply(lambda x: x.rename(columns=cols))
df.df_away = df.df_away.apply(lambda x: x.rename(columns=cols))

In [5]:
# restructure pbp to df and add column names

pbp_cols = ['time','home_events','score','away_events']

df.pbp = df.pbp.apply(lambda x: pd.DataFrame(x,columns=pbp_cols))

In [6]:
all_players = []

for row in df.iterrows():
  all_players += row[1][2]['name'].values.tolist()
  all_players += row[1][3]['name'].values.tolist()

unique_players = set(all_players)

player_keys = dict(zip(unique_players,range(0,len(unique_players))))

In [7]:
all_teams = []

all_teams += df.home.values.tolist()
all_teams += df.away.values.tolist()

unique_teams = set(all_teams)
team_keys = dict(zip(unique_teams,range(0,len(unique_teams))))

In [8]:
df = df.drop('df_sum',axis=1)

In [9]:
rosters = {}

for team in unique_teams:

  teamdf = df[df['home']==team]

  all_players = []

  for row in teamdf.iterrows():

    all_players += row[1][1]['name'].values.tolist()

  unique_players = set(all_players)


  teamdf = df[df['away']==team]

  all_players = []

  for row in teamdf.iterrows():
    all_players += row[1][2]['name'].values.tolist()


  all_players = set(all_players)

  rosters.update({team:all_players})

rosters

roster_data = []
for key, values in rosters.items():
    for value in values:
        roster_data.append({'team_name': key, 'player_name': value})

player_table = pd.DataFrame(roster_data)

In [10]:
player_table['player_ID'] = player_table['player_name'].map(player_keys)
player_table['team_ID'] = player_table['team_name'].map(team_keys)

In [11]:
box_table = pd.DataFrame()

for i,row in enumerate(df.iterrows()):
  temp_df = pd.DataFrame()
  temp_df = row[1][1]
  temp_df['team_name'] = row[1][4]
  temp_df['opposition_name'] = row[1][5]
  temp_df['team_ID'] = team_keys[row[1][4]]
  temp_df['opposition_ID'] = team_keys[row[1][5]]
  temp_df['player_ID'] = row[1][1]['name'].map(player_keys)
  temp_df['game_ID'] = i
  temp_df = temp_df.rename(columns = {'name':'player_name'})
  box_table = pd.concat([box_table,temp_df])

for i,row in enumerate(df.iterrows()):
  temp_df = pd.DataFrame()
  temp_df = row[1][2]
  temp_df['team_name'] = row[1][5]
  temp_df['opposition_name'] = row[1][4]
  temp_df['team_ID'] = team_keys[row[1][5]]
  temp_df['opposition_ID'] = team_keys[row[1][4]]
  temp_df['player_ID'] = row[1][2]['name'].map(player_keys)
  temp_df['game_ID'] = i
  temp_df = temp_df.rename(columns = {'name':'player_name'})
  box_table = pd.concat([box_table,temp_df])

box_table = box_table.drop([20,21],axis=1)

In [12]:
for col in box_table.loc[:,'goals':'ggs']:
    # Remove forward slash and convert string to float
    box_table[col] = box_table[col].str.replace('/', '').replace('', np.NaN).astype(float)

for col in box_table.loc[:,'ga':'dsv']:
    # Remove forward slash and convert string to float
    box_table[col] = box_table[col].str.replace('/', '').replace('', np.NaN).astype(float)

In [13]:
pbp_table = pd.DataFrame()

for i,row in tqdm(enumerate(df.iterrows())):
  temp_df = pd.DataFrame()
  temp_df = row[1][3]
  temp_df['home_team_name'] = row[1][4]
  temp_df['home_team_ID'] = team_keys[row[1][4]]
  temp_df['away_team_name'] = row[1][5]
  temp_df['away_team_ID'] = team_keys[row[1][5]]

  temp_df['game_ID'] = i
  pbp_table = pd.concat([pbp_table,temp_df])

773it [00:02, 356.29it/s] 


In [14]:
pbp_table['home_score'] = pbp_table.score.str[2]
pbp_table['away_score'] = pbp_table.score.str[0]
pbp_table = pbp_table.drop('score',axis = 1)

In [15]:
pbp_table['events'] = np.where(pbp_table['home_events'] == "",pbp_table['away_events'],pbp_table['home_events'])
pbp_table['event_team'] = np.where(pbp_table['home_events'] == "",'away','home')

pbp_table = pbp_table.drop(['home_events','away_events'],axis=1)

In [16]:
pbp_table['cat_event'] = np.where(pbp_table['events'].str.contains('GOAL'),'goal',
                                    np.where(pbp_table['events'].str.contains('Shot'),'shot',
                                      np.where(pbp_table['events'].str.contains('Foul'),'Foul_against',
                                        np.where(pbp_table['events'].str.contains('Penalty'),'corner',
                                          np.where(pbp_table['events'].str.contains('EMPTY'),'empty_net',
                                            np.where(pbp_table['events'].str.contains('STROKE'),'stroke_for',None))))))

In [17]:
pbp_table['shot_outcome'] = np.where(pbp_table['events'].str.contains('GOAL'),'goal',
                                    np.where((pbp_table['events'].str.contains('Shot')) & (pbp_table['events'].str.contains('WIDE')),'wide',
                                      np.where((pbp_table['events'].str.contains('Shot')) & (pbp_table['events'].str.contains('SAVE')),'save',
                                        np.where((pbp_table['events'].str.contains('Shot')) & (pbp_table['events'].str.contains('BLOCKED')),'blocked',
                                          np.where((pbp_table['events'].str.contains('Shot')) & (pbp_table['events'].str.contains('HIGH')),'wide',
                                            np.where((pbp_table['events'].str.contains('Shot')) & (pbp_table['events'].str.contains('Save')),'save',None))))))

In [18]:
pbp_table.time = pbp_table.time.replace('', np.NaN)
pbp_table.time = np.where(~pbp_table.time.isnull(), pbp_table.time.apply(lambda x: f'00:{x}'), pbp_table.time)
pbp_table.time = pd.to_timedelta(pbp_table.time)
pbp_table['time'] = pbp_table['time'].apply(lambda x: x.total_seconds() / 60)

In [19]:
schedule_lst = []
n = 0
for team in unique_teams:
  
  lst = df[df.home == team]['away'].tolist()

  lst += df[df.away == team]['home'].tolist()

  for i in lst:
    constructor = [team,i,n]
    schedule_lst.append(constructor)
    n+=1
schedule_table = pd.DataFrame(schedule_lst,columns=['team_name','opposition','game_ID'])

In [22]:
### save all tables to pkl
export = [schedule_table,pbp_table,box_table,player_table]

with open('/Users/euan_brown/Documents/GitHub/NCAA-fh-project/tables.pkl', 'wb') as f:
    pickle.dump(export, f)